In [136]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb

# Tensorflow

#### Config

In [137]:

config = {
    "learning_rate": 1e-3,
    "epochs": 100,
    "batch_size": 64,
    "optimizer": "Adam",
    "dropout_rate": 0.4
}

### WandB connection

In [138]:

wandb.login()
wandb.init(project="assignment-1", name="tensorflow-64-d-32-1", reinit=True, config=config)

### Loading the data

In [139]:
x_train = pd.read_csv('data/train/data.csv').to_numpy(dtype='float32')
y_train = pd.read_csv('data/train/labels.csv').to_numpy(dtype='float32')

x_val = pd.read_csv('data/val/data.csv').to_numpy(dtype='float32')
y_val = pd.read_csv('data/val/labels.csv').to_numpy(dtype='float32')

print(x_train[0])
print(y_train[0])
print()
print(x_val[0])
print(y_val[0])

[2.         1.         0.33221272 1.         1.         3.5115454 ]
[1.]

[ 3.         0.        -0.6710254  0.         0.         2.180892 ]
[0.]


### Building a model

In [140]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(rate=config['dropout_rate']),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),    
])

In [141]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 64)                448       
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_34 (Dense)            (None, 32)                2080      
                                                                 
 dense_35 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,561
Trainable params: 2,561
Non-trainable params: 0
_________________________________________________________________


In [142]:
if config['optimizer'] == 'SGD':
    optimizer = tf.keras.optimizers.SGD(learning_rate=config['learning_rate'])
elif config['optimizer'] == 'Adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [143]:
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        #return 1 / (1 + config['decay_rate'] * epoch) * config['learning_rate']\n",
        return (0.99 ** epoch) * config['learning_rate']

scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

### Training the model

In [144]:
history=model.fit(
    x_train,
    y_train,
    epochs=config['epochs'],
    validation_data=(x_val, y_val),
    batch_size=config['batch_size'],
    shuffle=True,
    callbacks=[
        scheduler_callback,
        wandb.keras.WandbCallback()
    ]
)

Epoch 1/100
 8/10 [=======================>......] - ETA: 0s - loss: 0.6747 - accuracy: 0.5879 INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best)... Done. 0.0s


10/10 [==============================] - 4s 356ms/step - loss: 0.6802 - accuracy: 0.5770 - val_loss: 0.6316 - val_accuracy: 0.6340 - lr: 0.0010
Epoch 2/100
 1/10 [==>...........................] - ETA: 0s - loss: 0.6658 - accuracy: 0.5938INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best)... Done. 0.0s


10/10 [==============================] - 3s 320ms/step - loss: 0.6542 - accuracy: 0.6402 - val_loss: 0.6180 - val_accuracy: 0.6302 - lr: 0.0010
Epoch 3/100
 1/10 [==>...........................] - ETA: 0s - loss: 0.5994 - accuracy: 0.7188INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best\assets


...

10/10 [==============================] - 2s 250ms/step - loss: 0.4131 - accuracy: 0.8331 - val_loss: 0.4313 - val_accuracy: 0.8226 - lr: 3.7724e-04
Epoch 99/100
 1/10 [==>...........................] - ETA: 0s - loss: 0.3700 - accuracy: 0.8750INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best)... Done. 0.0s


10/10 [==============================] - 2s 266ms/step - loss: 0.4221 - accuracy: 0.8201 - val_loss: 0.4310 - val_accuracy: 0.8264 - lr: 3.7346e-04
Epoch 100/100
 1/10 [==>...........................] - ETA: 0s - loss: 0.4963 - accuracy: 0.7969INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230325_185905-j7y32lxy\files\model-best)... Done. 0.0s


10/10 [==============================] - 2s 232ms/step - loss: 0.4286 - accuracy: 0.8266 - val_loss: 0.4308 - val_accuracy: 0.8264 - lr: 3.6973e-04


In [145]:
import plotly.express as px
fig = px.line({'loss': history.history['loss'], 'val_loss': history.history['val_loss']})
fig.show()

In [146]:
import plotly.express as px
fig = px.line({'accuracy': history.history['accuracy'], 'val_accuracy': history.history['val_accuracy']})
fig.show()

In [147]:
wandb.finish()

accuracy,▁▃▅▅▅▆▆▆▇▇▇▇▇█▇▇▇▇▇███▇▇█████▇██▇███▇█▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▂▂▂▂
lr,████▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▄▄▅▆▇▇▇▇██▇▇██████████████████████████
val_loss,█▇▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.82658
best_epoch,99
best_val_loss,0.43081
epoch,99
loss,0.42864
